In [7]:
import pandas as pd
from transformers import Wav2Vec2ForCTC
import torch


In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# Load the pre-trained model
model = Wav2Vec2ForCTC.from_pretrained('wav2vec2_model')

# Load the processor
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-large-960h')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

c:\Users\prave\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at wav2vec2_model and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [2]:
print(device)

cuda


In [4]:
import pandas as pd

# Load the CSV file to inspect it
class_map_df = pd.read_csv('yamnet_class_map.csv')

# Print the column names and the first few rows
print(class_map_df.columns)
print(class_map_df.head())

Index(['index', 'mid', 'display_name'], dtype='object')
   index        mid                display_name
0      0   /m/09x0r                      Speech
1      1   /m/0ytgt  Child speech, kid speaking
2      2  /m/01h8n0                Conversation
3      3  /m/02qldy        Narration, monologue
4      4  /m/0261r1                    Babbling


In [7]:
class_map = dict(zip(class_map_df['display_name'], class_map_df['mid']))


In [2]:
import os
import librosa
import pandas as pd
from torch.utils.data import Dataset

# Load the class mapping CSV file
class_map_df = pd.read_csv('yamnet_class_map.csv')
class_map = dict(zip(class_map_df['display_name'], class_map_df['mid']))

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map):
        # Load the ontology data (audio categories and their URLs)
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory

        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                for idx, url in enumerate(category["positive_examples"]):
                    # Assuming you have a consistent filename pattern
                    file_name = f"{category['name']}_{idx}.wav"
                    file_path = os.path.join(self.audio_directory, file_name)
                    file_path = file_path.replace("\\", "/")
                    
                    # Check if file exists in the directory
                    if os.path.exists(file_path):
                        audio_data = self.load_audio(file_path)

                        # Assign the label based on the class map
                        label = self.class_map.get(category['name'], -1)  # Default to -1 if not found
                        data.append({"audio": audio_data, "label": label})
                    else:
                        print(f"Audio file not found: {file_path}")
        
        return data

    def load_audio(self, file_path):
        # Load the audio file at a 16kHz sample rate
        audio, _ = librosa.load(file_path, sr=16000)
        return audio

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [12]:
import os
import librosa
import json
import pandas as pd
from torch.utils.data import Dataset
from pydub import AudioSegment
from pydub.utils import which  
# Explicitly set FFmpeg path for pydub
AudioSegment.ffmpeg = which("ffmpeg")  # This will check your system's PATH

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map):
        # Load the ontology data (audio categories and their URLs)
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory

        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                for idx, url in enumerate(category["positive_examples"]):
                    # Assuming you have a consistent filename pattern
                    file_name = f"{category['name']}_{idx}.mp3"
                    file_path = os.path.join(self.audio_directory, file_name)
                    
                    # Replace backslashes with forward slashes (for consistency)
                    file_path = file_path.replace("\\", "/")

                    # Debugging the file path
                    print(f"Checking MP3 file path: {file_path}")

                    # Check if file exists in the directory
                    if os.path.isfile(file_path):
                        # Convert MP3 to WAV if the file is in MP3 format
                        wav_file_path = file_path.replace(".mp3", ".wav")
                        print(f"Checking WAV file path: {wav_file_path}")
                        
                        if not os.path.isfile(wav_file_path):
                            print(f"Converting {file_path} to {wav_file_path}")
                            self.convert_mp3_to_wav(file_path, wav_file_path)
                        else:
                            print(f"WAV file already exists: {wav_file_path}")

                        # Load the converted or original WAV file
                        audio_data = self.load_audio(wav_file_path)

                        # Assign the label based on the class map
                        label = self.class_map.get(category['name'], -1)  # Default to -1 if not found
                        data.append({"audio": audio_data, "label": label})
                    else:
                        print(f"MP3 file not found: {file_path}")
        
        return data

    def convert_mp3_to_wav(self, mp3_file_path, wav_file_path):
        """Convert MP3 file to WAV."""
        try:
            audio = AudioSegment.from_mp3(mp3_file_path)
            audio.export(wav_file_path, format="wav")
            print(f"Converted {mp3_file_path} to {wav_file_path}")
        except Exception as e:
            print(f"Error converting {mp3_file_path} to WAV: {e}")

    def load_audio(self, file_path):
        """Load audio from a file (WAV format expected)."""
        try:
            audio, _ = librosa.load(file_path, sr=16000)
            return audio
        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [13]:
os.path.exists("audiosets\ontology\Animal_0.mp3")

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\prave\AppData\Local\Temp\ipykernel_24496\2044366985.py:1: SyntaxWarning: invalid escape sequence '\o'
  os.path.exists("audiosets\ontology\Animal_0.mp3")


True

In [ ]:
import os
import librosa
import json
import pandas as pd
from torch.utils.data import Dataset
from pydub import AudioSegment
from pydub.utils import which
from pydub import AudioSegment

# Set the explicit path to your FFmpeg executable
#AudioSegment.ffmpeg = r"C:\Program Files\ffmpeg-7.1-full_build-shared\bin\ffmpeg.exe"
print(f"Using FFmpeg from: {AudioSegment.ffmpeg}")

# Explicitly set FFmpeg path for pydub
#AudioSegment.ffmpeg = which("ffmpeg")  # This will check your system's PATH

import os
import json
import numpy as np
import ffmpeg
import pandas as pd
from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map):
        # Load the ontology data (audio categories and their URLs)
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory

        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for category in self.ontology_data:
            if "positive_examples" in category:
                for idx, url in enumerate(category["positive_examples"]):
                    # Assuming you have a consistent filename pattern
                    file_name = f"{category['name']}_{idx}.mp3"
                    file_path = os.path.join(self.audio_directory, file_name)

                    # Replace backslashes with forward slashes (for consistency)
                    file_path = file_path.replace("\\", "/")

                    # Debugging the file path
                    print(f"Checking MP3 file path: {file_path}")

                    # Check if file exists in the directory
                    if os.path.isfile(file_path):
                        # Convert MP3 to WAV if the file is in MP3 format
                        wav_file_path = file_path.replace(".mp3", ".wav")
                        print(f"Checking WAV file path: {wav_file_path}")

                        if not os.path.isfile(wav_file_path):
                            print(f"Converting {file_path} to {wav_file_path}")
                            self.convert_mp3_to_wav(file_path, wav_file_path)
                        else:
                            print(f"WAV file already exists: {wav_file_path}")

                        # Load the converted or original WAV file
                        audio_data = self.load_audio(wav_file_path)

                        # Assign the label based on the class map
                        label = self.class_map.get(category['name'], -1)  # Default to -1 if not found
                        data.append({"audio": audio_data, "label": label})
                    else:
                        print(f"MP3 file not found: {file_path}")
        
        return data

    def convert_mp3_to_wav(self, mp3_file_path, wav_file_path):
        """Convert MP3 file to WAV using ffmpeg-python."""
        try:
            # Check if the MP3 file exists
            if not os.path.isfile(mp3_file_path):
                raise FileNotFoundError(f"MP3 file not found: {mp3_file_path}")

            # Use ffmpeg to convert the MP3 to WAV
            ffmpeg.input(mp3_file_path).output(wav_file_path).run()
            print(f"Converted {mp3_file_path} to {wav_file_path}")
        except Exception as e:
            print(f"Error converting {mp3_file_path} to WAV: {e}")

    def load_audio(self, file_path):
        """Load audio from a file (WAV format expected) using ffmpeg-python."""
        try:
            # Check if the WAV file exists before loading
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Use ffmpeg to load the audio file
            out, _ = ffmpeg.input(file_path).output('pipe:1', format='wav').run(capture_stdout=True, capture_stderr=True)

            # Convert the bytes to a numpy array
            audio_data = np.frombuffer(out, np.int16)
            return audio_data
        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]



In [1]:
import os
import json
import numpy as np
import ffmpeg
import pandas as pd
from torch.utils.data import Dataset
from concurrent.futures import ThreadPoolExecutor, as_completed

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map, num_threads=4):
        # Load the ontology data (audio categories and their URLs)
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory
        self.num_threads = num_threads

        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        futures = []

        # Create a thread pool for concurrent processing
        with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            for category in self.ontology_data:
                if "positive_examples" in category:
                    for idx, url in enumerate(category["positive_examples"]):
                        # Assuming you have a consistent filename pattern
                        file_name = f"{category['name']}_{idx}.mp3"
                        file_path = os.path.join(self.audio_directory, file_name)

                        # Replace backslashes with forward slashes (for consistency)
                        file_path = file_path.replace("\\", "/")

                        # Debugging the file path
                        print(f"Checking MP3 file path: {file_path}")

                        # Check if file exists in the directory
                        if os.path.isfile(file_path):
                            # Convert MP3 to WAV if the file is in MP3 format
                            wav_file_path = file_path.replace(".mp3", ".wav")
                            print(f"Checking WAV file path: {wav_file_path}")

                            # Submit the conversion task to the executor
                            futures.append(executor.submit(self.handle_audio_file, file_path, wav_file_path, category, data, idx))
                        else:
                            print(f"MP3 file not found: {file_path}")

            # Wait for all tasks to complete
            for future in as_completed(futures):
                pass

        return data

    def handle_audio_file(self, mp3_file_path, wav_file_path, category, data, idx):
        """Handles the conversion and loading of audio in separate threads."""
        try:
            # Convert MP3 to WAV if necessary
            if not os.path.isfile(wav_file_path):
                print(f"Converting {mp3_file_path} to {wav_file_path}")
                self.convert_mp3_to_wav(mp3_file_path, wav_file_path)
            else:
                print(f"WAV file already exists: {wav_file_path}")

            # Load the converted or original WAV file
            audio_data = self.load_audio(wav_file_path)

            # Assign the label based on the class map
            label = self.class_map.get(category['name'], -1)  # Default to -1 if not found
            data.append({"audio": audio_data, "label": label})

        except Exception as e:
            print(f"Error handling {mp3_file_path}: {e}")

    def convert_mp3_to_wav(self, mp3_file_path, wav_file_path):
        """Convert MP3 file to WAV using ffmpeg-python."""
        try:
            # Check if the MP3 file exists
            if not os.path.isfile(mp3_file_path):
                raise FileNotFoundError(f"MP3 file not found: {mp3_file_path}")

            # Use ffmpeg to convert the MP3 to WAV
            ffmpeg.input(mp3_file_path).output(wav_file_path).run()
            print(f"Converted {mp3_file_path} to {wav_file_path}")
        except Exception as e:
            print(f"Error converting {mp3_file_path} to WAV: {e}")

    def load_audio(self, file_path):
        """Load audio from a file (WAV format expected) using ffmpeg-python."""
        try:
            # Check if the WAV file exists before loading
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Use ffmpeg to load the audio file
            out, _ = ffmpeg.input(file_path).output('pipe:1', format='wav').run(capture_stdout=True, capture_stderr=True)

            # Convert the bytes to a numpy array
            audio_data = np.frombuffer(out, np.int16)
            return audio_data
        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [3]:
import os
import json
import numpy as np
import ffmpeg
import pandas as pd
from torch.utils.data import Dataset
from multiprocessing import Pool

class AudioDataset(Dataset):
    def __init__(self, audio_directory, ontology_file, class_map, num_processes=4):
        # Load the ontology data (audio categories and their URLs)
        with open(ontology_file, 'r') as f:
            self.ontology_data = json.load(f)

        self.class_map = class_map
        self.audio_directory = audio_directory
        self.num_processes = num_processes

        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        tasks = []

        # Collect all tasks (files to process)
        for category in self.ontology_data:
            if "positive_examples" in category:
                for idx, url in enumerate(category["positive_examples"]):
                    file_name = f"{category['name']}_{idx}.mp3"
                    file_path = os.path.join(self.audio_directory, file_name)
                    file_path = file_path.replace("\\", "/")  # Ensure consistent paths
                    
                    if os.path.isfile(file_path):
                        wav_file_path = file_path.replace(".mp3", ".wav")
                        tasks.append((file_path, wav_file_path, category, data, idx))

        # Use multiprocessing Pool to process the tasks in parallel
        with Pool(processes=self.num_processes) as pool:
            pool.map(self.handle_audio_file, tasks)

        return data

    def handle_audio_file(self, task):
        """Handles the conversion and loading of audio in separate processes."""
        mp3_file_path, wav_file_path, category, data, idx = task
        try:
            if not os.path.isfile(wav_file_path):
                print(f"Converting {mp3_file_path} to {wav_file_path}")
                self.convert_mp3_to_wav(mp3_file_path, wav_file_path)
            else:
                print(f"WAV file already exists: {wav_file_path}")

            # Load the converted or original WAV file
            audio_data = self.load_audio(wav_file_path)

            # Assign the label based on the class map
            label = self.class_map.get(category['name'], -1)  # Default to -1 if not found
            data.append({"audio": audio_data, "label": label})

        except Exception as e:
            print(f"Error handling {mp3_file_path}: {e}")

    def convert_mp3_to_wav(self, mp3_file_path, wav_file_path):
        """Convert MP3 file to WAV using ffmpeg-python."""
        try:
            if not os.path.isfile(mp3_file_path):
                raise FileNotFoundError(f"MP3 file not found: {mp3_file_path}")

            # Use ffmpeg to convert the MP3 to WAV
            ffmpeg.input(mp3_file_path).output(wav_file_path).run()
            print(f"Converted {mp3_file_path} to {wav_file_path}")
        except Exception as e:
            print(f"Error converting {mp3_file_path} to WAV: {e}")

    def load_audio(self, file_path):
        """Load audio from a file (WAV format expected) using ffmpeg-python."""
        try:
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"WAV file not found: {file_path}")

            # Use ffmpeg to load the audio file
            out, _ = ffmpeg.input(file_path).output('pipe:1', format='wav').run(capture_stdout=True, capture_stderr=True)

            # Convert the bytes to a numpy array
            audio_data = np.frombuffer(out, np.int16)
            return audio_data
        except Exception as e:
            print(f"Error loading audio file {file_path}: {e}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [ ]:
from transformers import Trainer, TrainingArguments
import json  # Add this import statement

# Initialize the dataset (replace with the correct paths)
audio_directory = r"audiosets/ontology"  # Directory where audio files are stored
ontology_file = 'ontology.json'  # Path to the ontology file
class_map = pd.read_csv('yamnet_class_map.csv')

dataset = AudioDataset(audio_directory, ontology_file, class_map)
print("hello chunni training arrambikalam")
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    evaluation_strategy="epoch",     # evaluation strategy
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    logging_dir="./logs",            # directory for storing logs
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the model to be fine-tuned
    args=training_args,                  # training arguments
    train_dataset=dataset,               # the training dataset
    eval_dataset=dataset,                # evaluation dataset (you can use a separate dataset for validation)
)

# Start fine-tuning
trainer.train()
